In [39]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time

In [2]:
# Get Past Team Schedules
def getRegularSeasonSchedule(seasonYear,teamID):
    season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
    teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id = '00',season = season ,
                                                                  season_type_all_star='Regular Season',
                                                                  team_id = teamID).get_normalized_json()

    teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])
    teamGames['SEASON'] = season
    return teamGames

In [3]:
# Format Data Frame Columns and Get Home and Away Team IDs

def getGameDate(matchup):
    return matchup.partition(' at')[0][:10]

def getHomeTeam(matchup):
    return matchup.partition(' at')[2]

def getAwayTeam(matchup):
    return matchup.partition(' at')[0][10:]

def getTeamIDFromNickname(nickname):
    return teams.loc[teams['nickname'] == difflib.get_close_matches(nickname,teams['nickname'],1)[0]].values[0][0] 


In [4]:
def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):


    def getGameStats(teamID,gameID,seasonYear):
        #season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
        gameStats = cumestatsteam.CumeStatsTeam(game_ids=gameID,league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    data.at[1,'NICKNAME'] = awayTeamNickname
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [5]:
# Get NBA Team IDs
teams = pd.DataFrame(teams.get_teams())
teams.head(10)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [6]:
# Get Schedules for the last 2 seasons + current
scheduleFrame = pd.DataFrame()
years = [2020,2021,2022]

start = time.perf_counter_ns()
for year in years:
    for id in teams['id']:
        time.sleep(1)
        scheduleFrame = scheduleFrame.append(getRegularSeasonSchedule(year,id))
end = time.perf_counter_ns()

mins = ((end - start) / 1e9)/60
print(mins)

2.4451183616666667


In [7]:
scheduleFrame.head(10)

,MATCHUP,GAME_ID,SEASON
0,05/16/2021 Rockets at Hawks,0022001066,2020-21
1,05/13/2021 Magic at Hawks,0022001049,2020-21
2,05/12/2021 Wizards at Hawks,0022001042,2020-21
3,05/10/2021 Wizards at Hawks,0022001026,2020-21
4,05/06/2021 Hawks at Pacers,0022001000,2020-21
5,05/05/2021 Suns at Hawks,0022000992,2020-21
6,05/03/2021 Trail Blazers at Hawks,0022000978,2020-21
7,05/01/2021 Bulls at Hawks,0022000959,2020-21
8,04/30/2021 Hawks at 76ers,0022000949,2020-21
9,04/28/2021 Hawks at 76ers,0022000934,2020-21


In [8]:
# Identify Home and Away Teams and format date field

start = time.perf_counter_ns()
scheduleFrame['GAME_DATE'] = pd.to_datetime(scheduleFrame['MATCHUP'].map(getGameDate))
scheduleFrame['HOME_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getHomeTeam)
scheduleFrame['HOME_TEAM_ID'] = scheduleFrame['HOME_TEAM_NICKNAME'].map(getTeamIDFromNickname)
scheduleFrame['AWAY_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getAwayTeam)
scheduleFrame['AWAY_TEAM_ID'] = scheduleFrame['AWAY_TEAM_NICKNAME'].map(getTeamIDFromNickname)
scheduleFrame = scheduleFrame.drop_duplicates() # There's a row for both teams, only need 1
scheduleFrame = scheduleFrame.reset_index(drop=True)
end = time.perf_counter_ns()
secs = ((end - start) / 1e9)
print(secs)

6.0619276


In [9]:
getSingleGameMetrics(scheduleFrame.at[1,'GAME_ID'],scheduleFrame.at[1,'HOME_TEAM_ID'],
                     scheduleFrame.at[1,'AWAY_TEAM_ID'],scheduleFrame.at[1,'AWAY_TEAM_NICKNAME'],
                     scheduleFrame.at[1,'SEASON'],scheduleFrame.at[1,'GAME_DATE'])

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,BLK,PTS,AVG_REB,AVG_PTS,DQ,OFFENSIVE_EFFICIENCY,SCORING_MARGIN,SEASON,GAME_DATE,GAME_ID
0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,4,116,58.0,116.0,0,0.555556,23.0,2020-21,2021-05-13,0022001049
1,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,0,...,6,93,62.0,93.0,0,0.517241,-23.0,2020-21,2021-05-13,0022001049


In [10]:
gameLogs = pd.DataFrame()

In [14]:
i = int(len(gameLogs)/2) #where we last left off, will be 0 if just starting

while i<len(scheduleFrame):
    #Need to add a try except, try 5 times until logging failed instance and continue
    gameLogs =  gameLogs.append(getSingleGameMetrics(scheduleFrame.at[i,'GAME_ID'],scheduleFrame.at[i,'HOME_TEAM_ID'],
                     scheduleFrame.at[i,'AWAY_TEAM_ID'],scheduleFrame.at[i,'AWAY_TEAM_NICKNAME'],
                     scheduleFrame.at[i,'SEASON'],scheduleFrame.at[i,'GAME_DATE']))
    
    i+=1
    

gameLogs = gameLogs.reset_index(drop=True)

In [15]:
gameLogs.to_csv('gameLogs.csv')

In [18]:
gameLogs = gameLogs.reset_index(drop=True)

In [20]:
gameLogs

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,BLK,PTS,AVG_REB,AVG_PTS,DQ,OFFENSIVE_EFFICIENCY,SCORING_MARGIN,SEASON,GAME_DATE,GAME_ID
0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,8,124,63.0,124.0,0,0.653226,29.0,2020-21,2021-05-16,0022001066
1,OPPONENTS,Rockets,1610612745,0,1,0,0,0,1,1,...,5,95,51.0,95.0,0,0.459016,-29.0,2020-21,2021-05-16,0022001066
2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,4,116,58.0,116.0,0,0.555556,23.0,2020-21,2021-05-13,0022001049
3,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,0,...,6,93,62.0,93.0,0,0.517241,-23.0,2020-21,2021-05-13,0022001049
4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,5,120,56.0,120.0,0,0.573913,4.0,2020-21,2021-05-12,0022001042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5765,OPPONENTS,Wizards,1610612764,1,0,0,0,1,0,1,...,6,108,58.0,108.0,0,0.594340,8.0,2022-23,2022-11-07,0022200144
5766,Washington,Wizards,1610612764,1,0,1,0,0,0,1,...,7,120,55.0,120.0,0,0.614035,21.0,2022-23,2022-10-25,0022200052
5767,OPPONENTS,Pistons,1610612765,0,1,0,0,0,1,0,...,1,99,48.0,99.0,0,0.470588,-21.0,2022-23,2022-10-25,0022200052
5768,Charlotte,Hornets,1610612766,0,1,0,1,0,0,0,...,6,134,65.0,134.0,1,0.552846,-7.0,2022-23,2022-12-14,0022200414


In [58]:
def getHomeAwayFlag(gameDF):
    gameDF['HOME_FLAG'] = np.where((gameDF['W_HOME']==1) | (gameDF['L_HOME']==1),1,0)
    gameDF['AWAY_FLAG'] = np.where((gameDF['W_ROAD']==1) | (gameDF['L_ROAD']==1),1,0)
    return gameDF 

def getTotalWinPctg(gameDF):
    gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID','SEASON'])['GAME_DATE'].rank(ascending=True)
    gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W'].cumsum()
    gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS']/gameDF['TOTAL_GAMES_PLAYED']
    return gameDF.drop(['TOTAL_GAMES_PLAYED','TOTAL_WINS'],axis=1)
    
def getHomeWinPctg(gameDF):
    gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_FLAG'].cumsum()
    gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_HOME'].cumsum()
    gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS']/gameDF['HOME_GAMES_PLAYED']
    return gameDF.drop(['HOME_GAMES_PLAYED','HOME_WINS'],axis=1)
    
def getAwayWinPctg(gameDF):
    gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_FLAG'].cumsum()
    gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_ROAD'].cumsum()
    gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS']/gameDF['AWAY_GAMES_PLAYED']
    return gameDF.drop(['AWAY_GAMES_PLAYED','AWAY_WINS'],axis=1)

In [55]:
gameLogs[(gameLogs['TEAM_ID'] == 1610612737 ) & (gameLogs['SEASON'] == '2020-21')].sort_values('GAME_DATE')

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,GAME_DATE,GAME_ID,TOTAL_GAMES_PLAYED,TOTAL_WINS,TOTAL_WIN_PCTG,HOME_FLAG,AWAY_FLAG,HOME_GAMES_PLAYED,HOME_WINS,HOME_WIN_PCTG
143,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,2,...,2020-12-23,0022000015,1.0,1,1.000000,0,1,0,0,NaN
141,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,0,...,2020-12-26,0022000021,2.0,2,1.000000,0,1,0,0,NaN
138,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2020-12-28,0022000041,3.0,3,1.000000,1,0,1,1,1.000000
137,OPPONENTS,Hawks,1610612737,0,1,0,0,0,1,0,...,2020-12-30,0022000057,4.0,3,0.750000,0,1,1,1,1.000000
135,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,2,...,2021-01-01,0022000072,5.0,4,0.800000,0,1,1,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,OPPONENTS,Hawks,1610612737,0,1,0,0,0,1,0,...,2021-05-06,0022001000,68.0,37,0.544118,0,1,32,21,0.656250
6,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2021-05-10,0022001026,69.0,38,0.550725,1,0,33,22,0.666667
4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2021-05-12,0022001042,70.0,39,0.557143,1,0,34,23,0.676471
2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2021-05-13,0022001049,71.0,40,0.563380,1,0,35,24,0.685714


In [62]:
gameLogs = getHomeWinPctg(gameLogs)
gameLogs = getAwayWinPctg(gameLogs)

In [63]:
gameLogs

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,SEASON,GAME_DATE,GAME_ID,TOTAL_GAMES_PLAYED,TOTAL_WINS,TOTAL_WIN_PCTG,HOME_FLAG,AWAY_FLAG,HOME_WIN_PCTG,AWAY_WIN_PCTG
0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2020-21,2021-05-16,0022001066,72.0,41,0.569444,1,0,0.694444,0.444444
1,OPPONENTS,Rockets,1610612745,0,1,0,0,0,1,1,...,2020-21,2021-05-16,0022001066,72.0,17,0.236111,0,1,0.250000,0.222222
2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2020-21,2021-05-13,0022001049,71.0,40,0.563380,1,0,0.685714,0.444444
3,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,0,...,2020-21,2021-05-13,0022001049,70.0,21,0.300000,0,1,0.305556,0.294118
4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2020-21,2021-05-12,0022001042,70.0,39,0.557143,1,0,0.676471,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5765,OPPONENTS,Wizards,1610612764,1,0,0,0,1,0,1,...,2022-23,2022-11-07,0022200144,11.0,5,0.454545,0,1,0.400000,0.500000
5766,Washington,Wizards,1610612764,1,0,1,0,0,0,1,...,2022-23,2022-10-25,0022200052,4.0,3,0.750000,1,0,1.000000,0.500000
5767,OPPONENTS,Pistons,1610612765,0,1,0,0,0,1,0,...,2022-23,2022-10-25,0022200052,4.0,1,0.250000,0,1,1.000000,0.000000
5768,Charlotte,Hornets,1610612766,0,1,0,1,0,0,0,...,2022-23,2022-12-14,0022200414,28.0,7,0.250000,1,0,0.285714,0.214286
